# Extracting requirements from the ECB Guide

This script is designed to process the JSON output of an SCE Guide converted using Docling and transform  
the structured requirements into a pandas DataFrame. The primary goal is to extract headings at different   
levels (Level 0 to Level 3) into separate columns and organize the associated text into single rows.   
Additionally, the script extracts paragraph numbers from the text for better indexing and fills missing  
numbers where necessary.

**Please note that this approach provides a starting point, which needs to be reviewed manually**

#### Steps in the Code

##### 1. **Extract Headings and Text**

- The `extract_headings_separate_columns` function parses the input JSON (`doc`) to identify headings  
  and classify them into hierarchical levels (Level 0 to Level 3).
- Associated text is grouped under the appropriate headings.


##### 2. **Flatten the Result**

- Headings and text are flattened into single rows to facilitate further transformations.


##### 3. **Create a Pandas DataFrame**

- The resulting data is stored in a DataFrame with clearly defined columns:  
  **Page**, **Level 0 to Level 3**, **Text**, **Label**, **Parent**, and a concatenated levels column.


##### 4. **Extract Paragraph Numbers**

- A helper function, `extract_number`, extracts numeric paragraph identifiers at the start of the text.  
  Missing numbers are forward-filled for consistency.


##### 5. **Join rows, which belong to the same paragraph**

- During the Docling processing some paragraphs are split and need to be joined again

In [1]:
from docling.document_converter import DocumentConverter
import json
import re
import pandas as pd

# Ensure no limitations in Jupyter Notebook output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Transform pdf
source = "ssm.supervisory_guides_202402.pdf"
converter = DocumentConverter()
result = converter.convert(source)
doc = result.document.export_to_dict()

# Save the dictionary to a JSON file
with open('eba_guide.json', 'w', encoding='utf-8') as json_file:
    json.dump(result.document.export_to_dict(), json_file, indent=4)

# Write the result to a markdown file
with open('eba_guide.md', 'w', encoding='utf-8') as f:
    f.write(result.document.export_to_markdown())

print("Document saved as JSON and markdown file!")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Document saved as JSON and markdown file!


In [3]:
def extract_headings_separate_columns(doc):
    result = []
    texts = doc.get('texts', [])

    # Variables to keep track of Level 0, Level 1, Level 2, and Level 3 headings
    current_level_0 = None
    current_level_1 = None
    current_level_2 = None
    current_row = None

    for text in texts:
        label = text.get('label', '')
        orig = text.get('orig', '')
        prov = text.get('prov', [])
        page_no = prov[0]['page_no'] if prov else None
        parent = text.get('parent', {}).get('$ref', None)

        exceptions = [
            'Risk differentiation across grades or pools',
            'Homogeneity within grades',
            'Distribution of obligors or facilities across grades or pools',
            'Principles specific for grades and pools',
            'Principles for all model types',
            'Principles specific for direct estimates',
            'For example:',
            '77. Validation policy'
        ]

        if orig in exceptions:
            label = 'text'
            
        if label == 'section_header':
            first_word = orig.split(' ', 1)[0]
            
            # Check if this is a Level 0 heading (no dots and starts with a capital letter)
            if '.' not in orig.split(' ', 1)[0] and orig[0].isupper() and not first_word.endswith('.'):
                current_level_0 = orig
                current_level_1 = None  # Reset Level 1 when a new Level 0 is found
                current_level_2 = None
                current_row = (page_no, orig, None, None, None, [], None, None)

            # Check if this is a Level 1 heading (no dot in numbering)
            elif orig.split(' ', 1)[0].isdigit():
                current_level_1 = orig
                current_level_2 = None  # Reset Level 2 when a new Level 1 is found
                current_row = (page_no, current_level_0, orig, None, None, [], None, None)

            # Check if this is a Level 2 heading (contains one dot in numbering)
            elif orig.split(' ', 1)[0].count('.') == 1 and not first_word.endswith('.'):
                current_level_2 = orig
                current_row = (page_no, current_level_0, current_level_1, orig, None, [], None, None)

            # Check if this is a Level 3 heading (contains two dots in numbering)
            elif orig.split(' ', 1)[0].count('.') == 2 and not first_word.endswith('.'):
                current_row = (page_no, current_level_0, current_level_1, current_level_2, orig, [], None, None)

            if current_row:
                result.append(current_row)

        # A simplification: consider including footnotes, tables and pictures
        elif label in ['list_item', 'text']:
            # Append list items (text under the heading) and label, parent to the "Text" field
            if current_row:
                current_row[-3].append((orig, label, parent))

    # Flatten the "Text" field for multiple rows
    final_result = []
    for row in result:
        page_no, level_0, level_1, level_2, level_3, texts, _, _ = row
        if texts:
            for text, label, parent in texts:
                final_result.append((page_no, level_0, level_1, level_2, level_3, text, label, parent))
        else:
            final_result.append((page_no, level_0, level_1, level_2, level_3, None, None, None))

    return final_result

# Extract headings into separate columns
headings_separated = extract_headings_separate_columns(doc)
df = pd.DataFrame(headings_separated, 
                  columns=["Page", "Level 0", "Level 1", "Level 2", "Level 3", "Text", "Label", "Parent"])
df["Concatenated Levels"] = df[["Level 0", "Level 1", "Level 2", "Level 3"]].apply(
    lambda x: " > ".join(filter(None, x)), axis=1
)

# Extract the number at the beginning of the "Text" column
def extract_number(text):
    if isinstance(text, str):  # Ensure the value is a string
        match = re.match(r"^(\d+)\.", text)
        return int(match.group(1)) if match else None
    return None

df['Par'] = df['Text'].apply(extract_number)

# Fill forward the missing numbers in 'Par'
df['Par'] = df['Par'].ffill()

In [4]:
# Remove rows with None values in Text
df = df[df['Text'].notna()]

# Function to determine if rows should be merged
def should_merge(prev_row, curr_row):
    # Check if Level 3 matches
    if prev_row['Level 3'] != curr_row['Level 3']:
        return False

    # Scenario 1: Merge if sentence ends without punctuation and new row starts without capital letter or punctuation
    if not prev_row['Text'].strip().endswith(('.')) and curr_row['Text'].strip()[0].islower():
        return True

    # Scenario 2: Merge if new row starts with (a), (b), (iv) etc.
    patterns = r"^\((?:[a-z]{1,2}|\d+|i{1,3}|iv|v)\)|^·"
    if re.match(patterns, curr_row['Text'].strip()):   
        curr_row['Text'] = '\n' + curr_row['Text']
        return True

    return False

# Process the DataFrame
merged_rows = []
temp_row = None

for index, row in df.iterrows():
    if temp_row is None:
        temp_row = row.copy()
    else:
        if should_merge(temp_row, row):
            temp_row['Text'] += ' ' + row['Text']
        else:
            merged_rows.append(temp_row)
            temp_row = row.copy()

# Add the last temp_row to the merged rows if not None
if temp_row is not None:
    merged_rows.append(temp_row)

# Create a new DataFrame from merged rows
merged_df = pd.DataFrame(merged_rows)

# Store the dataframe
merged_df.to_excel('ecb_guide.xlsx', index=False)

merged_df[500:530]

,Page,Level 0,Level 1,Level 2,Level 3,Text,Label,Parent,Concatenated Levels,Par
1128,142,Credit risk,8 Model-related MoC,8.1 Relevant regulatory references,None,"209. Since the MoC requirements laid down by the CRR also apply in cases where institutions estimate CCFs, paragraph 208 is also relevant in such cases.",list_item,#/groups/210,Credit risk > 8 Model-related MoC > 8.1 Relevant regulatory references,209.0
1129,142,Credit risk,8 Model-related MoC,8.1 Relevant regulatory references,None,"210. In the understanding of the ECB, to reflect the dispersion of the statistical estimators as set out in paragraph 43(b) of the EBA Guidelines on PD and LGD, institutions should adopt the following approach. \n(a) For PD, estimate an MoC to account for statistical uncertainty/sampling error affecting the LRA estimate at grade/pool level. This MoC should be based on the distribution of the estimator, which is the average of one-year default rates of the grade/pool across time (i.e. the distribution of (ΣDR$_{t}$) T / ), considering that the uncertainty is primarily driven by the statistical uncertainty of each one-year default rate and the length of the time series. As a result, it is expected that the lower the number of observations per grade and the shorter the time series are, the higher the MoC of the grade should be.",list_item,#/groups/211,Credit risk > 8 Model-related MoC > 8.1 Relevant regulatory references,210.0
1131,142,Credit risk,8 Model-related MoC,8.1 Relevant regulatory references,None,"Institutions need to be aware of and deal adequately with the dependency between default rates over time on the quantification of the MoC, e.g. when using overlapping windows for the calculation of default rates.",text,#/body,Credit risk > 8 Model-related MoC > 8.1 Relevant regulatory references,210.0
1132,142,Credit risk,8 Model-related MoC,8.1 Relevant regulatory references,None,"The above principles also apply for institutions using direct PD estimates and for institutions calibrating the LRA default rate at the level of the calibration segment, as referred to in paragraph 92(b) of the EBA Guidelines on PD and LGD. When using direct PD estimates, the MoC is based on the distribution of this direct PD estimator (which includes the risk differentiation function), implicitly reflecting the uncertainty of the LRA. When calibration is performed at calibration segment level, the general estimation error may be computed at that level when the statistical uncertainty/sampling error is neither significantly different across grades or PD sub-ranges nor significantly different between the calibration segment level and the grades or PD sub-ranges level. \n(b) Similarly, for LGD and CCF, estimate an MoC to account for statistical uncertainty/sampling error affecting the final estimates. This MoC should be defined on the basis of the distribution of the estimators, considering that their uncertainty is primarily driven by the statistical uncertainty of the observations used to compute the long-run and downturn estimates and the length of the time series.",text,#/body,Credit risk > 8 Model-related MoC > 8.1 Relevant regulatory references,210.0
1135,144,Credit risk,9 Review of estimates,9.1 Relevant regulatory references,None,"211. Institutions must review their estimates whenever new information comes to light but at least on an annual basis. 97 To comply with this requirement, they are expected to have in place a framework under paragraphs 217 to 221 of the EBA Guidelines on PD and LGD.",list_item,#/groups/213,Credit risk > 9 Review of estimates > 9.1 Relevant regulatory references,211.0
1136,144,Credit risk,9 Review of estimates,9.1 Relevant regulatory references,None,"212. Since the review of estimates requirements under the CRR also apply in cases where an institution estimates CCFs, paragraph 211 is also relevant to such cases.",list_item,#/groups/213,Credit risk > 9 Review of estimates > 9.1 Relevant regulatory references,212.0
1137,144,Credit 